**学号：** ________  &nbsp; **姓名：**________  &nbsp; **班级：**________  

# 【报告3】 聚类实验

对数据集`wholesale_customers.csv`进行聚类分析。

### 提交要求：
1. 填写Word版实验报告表格；
2. 提交jupyter笔记和该笔记PDF；
3. 文件名格式示例：
   * `学号-姓名-实验报告.docx`
   * `学号-姓名-实验笔记.ipynb`
   * `学号-姓名-实验笔记.pdf`

> 学委收齐，整理在一个文件夹内，打包发给我。

<br>

### 1. 数据读取与预处理


### 2. 聚类分析

### 3. 分类模型

将聚类的结果作为分类的目标，使用逻辑回归和随机森林模型建立分类模型，分析各特征重要程度。
